In [5]:
from splinter import Browser
from bs4 import BeautifulSoup
import numpy as np
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import time
import pandas as pd

app = Flask(__name__)

app.config["MONGO_URI"] = "mongodb://localhost:27017/articles"
mongo = PyMongo(app)

In [6]:
def init_browser():
    executable_path = {"executable_path": "C:/chromedriver/chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

In [7]:
#Python dictionary
site_data = {}